In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import warnings 
warnings.simplefilter("ignore")

In [2]:
UserID =3610000000000000000

In [3]:
articles_df=(pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/teaable2.csv'))

In [4]:
articles_df['contentId']=articles_df['품목']
articles_df

,차품목,효능번호,카페인,향기준,품목,"맛,향",contentId
0,Bengal Spice,1,X,2,a1,"계피 향, 생강 향",a1
1,BerryBlossom White,1,O,1,a2,베리 향,a2
2,Black Cherry Berry,3,X,1,a3,"체리 향, 베리 향",a3
3,Breakfast in Paris Black Tea,1,O,"1,4",a4,"라벤더꽃 향, 시트러스 향",a4
4,ButterScotch Blondie,3,O,4,a5,"버터카라멜 향, 바닐라 향",a5
...,...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",a66,"감초 향, 카다몬 향, 회향씨 향, 박하 향, 생강뿌리 향, 후추 향",a66
66,요기티 Yogi Tea 진저 티 16티백,"2,4",NaN,2,a67,허브 약초 생강,a67
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",a68,"생강뿌리 향, 감초 향, 유칼립투스잎 향, 오렌지껍질 향, 박하 향, 레몬그라스 향...",a68
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,a69,"누릅나무껍질 향, 꿀 향, 레몬 향",a69


In [5]:
articles_df['맛,향']=articles_df['맛,향'].str.replace('[향,' ',(,),]?' , '')
articles_df['contentId']=articles_df['contentId'].str.replace('a?','')
articles_df['카페인']=articles_df['카페인'].fillna('X')
articles_df.drop('품목',axis=1)

,차품목,효능번호,카페인,향기준,"맛,향",contentId
0,Bengal Spice,1,X,2,계피 생강,1
1,BerryBlossom White,1,O,1,베리,2
2,Black Cherry Berry,3,X,1,체리 베리,3
3,Breakfast in Paris Black Tea,1,O,"1,4",라벤더꽃 시트러스,4
4,ButterScotch Blondie,3,O,4,버터카라멜 바닐라,5
...,...,...,...,...,...,...
65,요기티 Yogi Tea 스토맥 이즈 티 16티백,4,X,"2,3",감초 카다몬 회씨 박하 생강뿌리 후추,66
66,요기티 Yogi Tea 진저 티 16티백,"2,4",X,2,허브 약초 생강,67
67,요기티 Yogi Tea 콜드 시즌 티 16티백,4,X,"2,3,4",생강뿌리 감초 유칼립투스잎 오렌지껍질 박하 레몬그라스 바질 카다몬 오...,68
68,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,4,X,1,누릅나무껍질 꿀 레몬,69


In [6]:
interaction_df = pd.read_csv('merge_outer.csv')
interaction_df['contentId']=interaction_df['contentId'].str.replace('a?','')
interaction_df['contentId'].astype(float)


0       23.0
1        1.0
2       30.0
3       66.0
4       58.0
        ... 
9141    57.0
9142    49.0
9143    36.0
9144    67.0
9145    24.0
Name: contentId, Length: 9146, dtype: float64

In [7]:
interaction_df=interaction_df.drop('Unnamed: 0',axis=1)


In [8]:
interaction_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,interactCnt,gender,age(15~60)
0,1463138398,VIEW,23,-9.220000e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
1,1463656314,VIEW,1,-9.220000e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
2,1464113091,VIEW,30,-9.220000e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
3,1462283851,VIEW,66,-9.220000e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
4,1462452127,VIEW,58,-9.220000e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
...,...,...,...,...,...,...,...,...,...,...,...
9141,1464955959,VIEW,57,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22
9142,1464955960,VIEW,49,9.200000e+18,-9.110000e+18,NaN,NaN,NaN,5,male,22
9143,1464956193,VIEW,36,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22
9144,1464955914,VIEW,67,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22


In [9]:
event_type_strength = {
   'VIEW': 1.0,
   'JJIM': 2.0, 
   'comment1': 1.0, 
   'comment2': 2.0,
   'comment3': 3.0,
    'comment4': 4.0, 'comment5': 5.0
}

interaction_df['eventStrength'] = (
  interaction_df
    .loc[:, 'eventType']
    .apply(lambda d: event_type_strength[d])
)

In [10]:
interaction_df_over5 = (interaction_df
  .groupby('personId', group_keys=False)
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5]
)
interaction_df_over5

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,interactCnt,gender,age(15~60),eventStrength
0,1463138398,VIEW,23,-9.220000e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29,1.0
1,1463656314,VIEW,1,-9.220000e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29,1.0
2,1464113091,VIEW,30,-9.220000e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29,1.0
3,1462283851,VIEW,66,-9.220000e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29,1.0
4,1462452127,VIEW,58,-9.220000e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9141,1464955959,VIEW,57,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22,1.0
9142,1464955960,VIEW,49,9.200000e+18,-9.110000e+18,NaN,NaN,NaN,5,male,22,1.0
9143,1464956193,VIEW,36,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22,1.0
9144,1464955914,VIEW,67,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22,1.0


In [11]:
interaction_full_df = (
  interaction_df_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength']))
)

interaction_full_df.head(10)

,personId,contentId,eventStrength,eventScore
0,-9.220000e+18,1,1.0,1.0
1,-9.220000e+18,23,1.0,1.0
2,-9.220000e+18,30,1.0,1.0
3,-9.220000e+18,58,1.0,1.0
4,-9.220000e+18,66,1.0,1.0
5,-9.170000e+18,14,1.0,1.0
6,-9.170000e+18,20,1.0,1.0
7,-9.170000e+18,21,1.0,1.0
8,-9.170000e+18,23,1.0,1.0
9,-9.170000e+18,4,1.0,1.0


# 평가

In [12]:
interaction_train, interaction_test = train_test_split(
    interaction_full_df,
    stratify=interaction_full_df['personId'],
    test_size=0.2,
    random_state=40
)

In [13]:
interaction_full_indexed = interaction_full_df.set_index('personId')
interaction_train_indexed = interaction_train.set_index('personId')
interaction_test_indexed = interaction_test.set_index('personId')


In [14]:
def get_items_interacted(person_id, interaction_df):
    interacted_items = interaction_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])
#get_items_interacted로 person_id와 입력받은 person_id의 contentId를 Series로 받아옴

In [15]:
class ModelEvaluator:
    def __init__(self, n_non_interacted=15): #여기서 우리는 티가 70개 밖에 되지 않음. 따라서 상호작용하지 않은 애들을 15개로 잡을 예정
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = n_non_interacted 
        #ModelEvaluator 실행시 가장 먼저 실행되는 부분
        #해당 인스턴스의 random_sample_non_interacted_items 설정
        
    def get_non_interacted_items_sample(self, person_id, sample_size, seed=40):
        interacted_items = get_items_interacted(person_id, interaction_full_indexed) #인덱싱 된 interaction_full_indexed를 가지고 
        #get_items_interacted 실행
        all_items = set(articles_df['contentId']) # 티의 content_id를 중복을 제외한 상태인 set집합으로 가져옴
        non_interacted_items = all_items - interacted_items #상호작용이 없었던 애들을 전체 집합에서 빼서 가져옴
        
        random.seed(seed) #랜덤시드 초기화
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size) #sample_size만큼의 non_interacted_items를 랜덤추출
        return set(non_interacted_items_sample) #중복을 제외한 애들을 샘플집합으로 가져옴
        
    def _verify_hit_top_n(self, item_id, recommend_items, topn):
        try:
            index = next(i for i, c in enumerate(recommend_items) if c == item_id)
        except:
            index = -1 
        hit = int(index in range(0, topn))
        return hit, index
    
    def evaluate_model_for_user(self, model, person_id):
        interacted_values_testset = interaction_test_indexed.loc[person_id] 
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
        #유저별 event한번이라도 발생한 item의 아이템셋 집합 가져오기
        interacted_items_count_testset = len(person_interacted_items_testset)
        #개수
        # 특정 사용자에 대한 추천 순위 목록을 받아온다
        person_recs = model.recommend_items(
            person_id,
            items_to_ignore=get_items_interacted(person_id, interaction_train_indexed),
            topn=10
        )
        
        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # test셋에서 사용자가 상호작용한 모든 항목에 대해 반복한다
        for item_id in person_interacted_items_testset:
            
            # 사용자가 상호작용하지 않은 5개 항목을 샘플링한다
            non_interacted_items_sample = self.get_non_interacted_items_sample(
                person_id,
                sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                seed=10
            )
            
            # 현재 선택한 item_id(상호작용 있었던 항목)와 15개 랜덤 샘플을 합친다
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))
            
            # 추천 결과물 중에서 현재 선택한 item_id와 5개 랜덤 샘플의 결과물로만 필터링한다
            valid_recs_df = person_recs[person_recs['contentId'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['contentId'].values
            
            # 현재 선택한 item_id가 Top-N 추천 결과 안에 있는지 확인한다
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
            
        # Recall 값은 상호작용 있었던 항목들 중에서 관련없는 항목들과 섞였을 때 Top-N에 오른 항목들의 비율로 나타낼 수 있다
        recall_at_5 = hits_at_5_count / interacted_items_count_testset
        recall_at_10 = hits_at_10_count / interacted_items_count_testset
        
        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'interacted_count': interacted_items_count_testset,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10
        }
        return person_metrics
    
    def evaluate_model(self, model):
        people_metrics = []
        for idx, person_id in enumerate(list(interaction_test_indexed.index.unique().values)):
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)

        print('{} users processed'.format(idx))
        
        detailed_result = (
            pd.DataFrame(people_metrics)
              .sort_values('interacted_count', ascending=False)
        )
        
        global_recall_at_5 = detailed_result['hits@5_count'].sum() / detailed_result['interacted_count'].sum()
        global_recall_at_10 = detailed_result['hits@10_count'].sum() / detailed_result['interacted_count'].sum()
        
        global_metrics = {
            'model_name': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10
        }
        
        return global_metrics, detailed_result

In [16]:
model_evaluator = ModelEvaluator(n_non_interacted=20)

In [17]:
import nltk
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=1000,
    
)

In [18]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=1000,
    
)

In [19]:
item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['맛,향'])
tfidf_feature_names = vectorizer.get_feature_names()

In [20]:
tfidf_matrix

<70x131 sparse matrix of type '<class 'numpy.float64'>'
	with 222 stored elements in Compressed Sparse Row format>

In [21]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id, interaction_indexed_df):
    interaction_person_df = interaction_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interaction_person_df['contentId'])
    
    user_item_strengths = np.array(interaction_person_df['eventStrength']).reshape(-1, 1)
    
    # 상호작용 강도를 바탕으로 가중치를 부여하여 평균 계산한다
    user_item_strengths_weighted_avg = \
        np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /\
        np.sum(user_item_strengths)
        
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_user_profiles():
    interaction_indexed_df = (interaction_full_df
        .loc[lambda d: d['contentId'].isin(articles_df['contentId'])]
        .set_index('personId')
    )
    user_profiles = {}
    
    for person_id in interaction_indexed_df.index.unique():
        user_profiles[person_id] = build_user_profile(person_id, interaction_indexed_df)
        
    return user_profiles

In [22]:
user_profiles = build_user_profiles()
len(user_profiles)

397

In [23]:

myprofile = user_profiles[UserID].flatten().tolist()
A= pd.DataFrame(sorted(zip(tfidf_feature_names, myprofile), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])
A

,token,relevance
0,복숭아,0.404285
1,계피,0.354735
2,바닐라,0.314185
3,민트,0.243004
4,캐모마일,0.234975
5,생강,0.225139
6,시트러스,0.190655
7,베리,0.188396
8,홍차,0.184345
9,레몬,0.165813


# 인기도 모형

In [24]:
item_popularity = (interaction_full_df
 .groupby('contentId')['eventStrength'].sum()
 .sort_values(ascending=False)
 .reset_index()
)

item_popularity.head(10)

,contentId,eventStrength
0,58,187.0
1,11,185.0
2,50,176.0
3,27,176.0
4,44,176.0
5,61,175.0
6,65,171.0
7,46,171.0
8,4,170.0
9,55,170.0


In [25]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 인기상품 중에서 사용자가 보지 않았던 상품을 추천한다
        recommendations = (
          self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)]
            .sort_values('eventStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['eventStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations

# ContentBasedRecommender

In [26]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, item_ids, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        # 유저 특성과 항목 특성 사이의 코사인 유사도를 구한다
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        
        # 가장 유사한 항목을 찾는다
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # 유사도를 기준으로 유사한 항목을 정렬한다
        similar_items = sorted(
            [(item_ids[i], cosine_similarities[0, i]) for i in similar_indices],
            key=lambda x: -x[1]
        )
        
        return similar_items
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        # 기존에 상호작용했던 항목은 제거한다
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations = (
            pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength'])
              .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                 raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations

In [27]:
content_based_model = ContentBasedRecommender(item_ids, articles_df)
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(content_based_model)
pop_detailed_results.head()

ValueError: Sample larger than population or is negative

In [ ]:
users_items_pivot_df = (interaction_train
  .pivot(index='personId', columns='contentId', values='eventStrength')
  .fillna(0)
)

In [ ]:
users_items_pivot_matrix = users_items_pivot_df.values
users_items_pivot_matrix[:10]

In [ ]:
NUMBER_OF_FACTORS_MF = 15
U, sigma, Vt = svds(users_items_pivot_matrix, k=NUMBER_OF_FACTORS_MF)
sigma_mat = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma_mat), Vt)
all_user_predicted_ratings

In [ ]:
user_ids = list(users_items_pivot_df.index)
user_ids[:10]

cf_preds_df = (
  pd.DataFrame(all_user_predicted_ratings, 
               columns=users_items_pivot_df.columns, 
               index=user_ids)
    .transpose()
)

# Collaborative Filtering

In [ ]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 사용자에 대한 예측값을 가져와서 정렬한다
        sorted_user_prediction = (self.cf_predictions_df
            .loc[:, user_id]
            .sort_values(ascending=False)
            .reset_index()
            .rename(columns={user_id: 'recStrength'})
        )
        
        recommendations = (sorted_user_prediction
            .loc[lambda d: ~d['contentId'].isin(items_to_ignore)]
            .sort_values('recStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.item_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations

In [ ]:
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

# Hybrid

In [ ]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 상위 1000개의 컨텐츠 기반 모형 추천을 가져온다
        cb_recs = (self.cb_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCB'})
        )
        
        # 상위 1000개의 협업필터링 추천을 가져온다
        cf_recs = (self.cf_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCF'})
        )
        
        # 1) 두 모형의 결과를 합친다
        # 2) CF, CB 모형의 점수를 바탕으로 Hybrid 모형의 점수를 계산한다
        # 3) Hybrid 점수를 기준으로 정렬한다
        recommendations = (cb_recs
            .merge(cf_recs, how='inner', left_on='contentId', right_on='contentId')
            .assign(recStrengthHybrid = lambda d: d['recStrengthCB'] * d['recStrengthCF'])
            .sort_values('recStrengthHybrid', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrengthHybrid', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations

In [ ]:
hybrid_recommender_model = HybridRecommender(content_based_model, cf_recommender_model, articles_df)